## 石头堆--> piles = [3, 9, 1, 2]
## 两个相同聪明的人轮流拿石头堆，只能从最左或最右取石头
## 设计算法，返回先手和后手最后得分之差

================================================================================================================

先建立dp二维数组,每一个元素都是一个元组,含有两个元素
例子：
* dp[0][1] 代表石头堆为piles[0:1+1]时，先手和后手最终获得的石头数 --> dp[0][1] = (3,0)
* dp[0][3] 代表石头堆为piles[0:3+1]的情况，这时便是所有的石头了，一眼看出9是肯定要拿的，所以先拿2，然后便是9 --> dp[0][3] = (11,4)

状态转移方程需要我们找到**所有“状态”**和**每个状态可以做的“选择”**，然后**择优**
根据dp数组的定义，**状态有三个**：开始的索引i，结束的索引j，当前轮到的人
* dp[i][j][fir or sec], 其中0<=i<len(piles) AND i<=j<len(piles)

In [ ]:
"""
对于这个问题的每一个状态，可以做的选择有两个：最左边的那堆石头或者最右边的那堆石头，我们穷举所有状态：
n = len(piles)
for i in range(n):
    for j in range(i, n):
        for who in {fir, sec}:
            dp[i][j]  = max(left, right)
首先这肯定不是最终答案，因为这只考虑了当前情况下最左和最右的最优解，而我们看到[3,9,1,2]时都会考虑下一步对方是否能拿到那个9
即，先手的选择对后手是有影响的

根据对dp数组的定义，写出状态转移方程:
dp[i][j].fir = max(piles[i] + dp[i+1][j].sec, piles[j] + dp[i][j-1].sec)
             = max(选择最左边的石头堆         , 选择最右边的石头堆         )
if 先手左边石头堆：
    dp[i][j].sec = dp[i+1][j].fir
elif 先手右边石头堆：
    dp[i][j].sec = dp[i][j-1].fir
这就把每一个数组的后手数值的计算迭代下去了

根据dp数组的定义，找出base case,即两个人面对最后一个石头堆时，这是存在的，因为石头堆总数可以是奇数也可以是偶数
dp[i][j].fir = piles[i]
dp[i][j].sec = 0
如下：
start\end    0     1     2     3
        0  (3,0)
        1        (9,0)
        2              (1,0)
        3                    (2,0)

我们推算dp[i][j].fir时其实用到了dp[i+1][j]和dp[i][j-1]，即下一行， 左一列
start\end    0     1     2     3
        0  (3,0)
        1        (9,0) (9,1)
        2              (1,0) (2,1)
        3                    (2,0)
dp[2][2].sec = dp[2][1].fir

所以要实现斜着遍历数组
"""

In [58]:
def diffStones(piles = [3,9,1,2]):
    import copy
    n = len(piles)
    dp = []
    for i in range(n):
        d = [[0,0] for f in range(i)]+[[piles[i], 0]] + [[0,0] for f in range(n-i-1)]
        dp.append(d)  
    ## 斜着遍历数组
    for diff in range(1,n):
        for i in range(n-diff):
            j = diff + i
            ##以上是实现斜边遍历的循环，可以手写草稿找到diff和i的范围
            ## 先手选择左边石头堆或右边石头堆的两个数值，择优
            left = piles[i] + dp[i+1][j][1]
            right= piles[j] + dp[i][j-1][1]
            #print(f"dp[{i}][{j}]", end = "\t")
            #print(f"left:{left}\tright:{right}", end = "\t")
            if left > right:
                dp[i][j][0] = left
                dp[i][j][1] = dp[i+1][j][0]
            else:
                dp[i][j][0] = right
                dp[i][j][1] = dp[i][j-1][0]
            #print(f"dp[{i}][{j}] = {dp[i][j]}")
            #print(dp[i])
    for i in dp:
        print(i)
    return abs(dp[0][n-1][0] - dp[0][n-1][1])

diffStones([3,9,1,2])

[[3, 0], [9, 3], [4, 9], [11, 4]]
[[0, 0], [9, 0], [9, 1], [10, 2]]
[[0, 0], [0, 0], [1, 0], [2, 1]]
[[0, 0], [0, 0], [0, 0], [2, 0]]


7

In [48]:
### deep copy示例
import copy
a = [[0]*10]
for i in range(9):
    a.append(copy.deepcopy(a[0]))
a[0][2] = 9
a

[[0, 0, 9, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [50]:
## 草拟
## 第一步，填入base case
## 这个list的创建真复杂， 存在shallow copy的情况
## 解决办法有二：
## 一是用for循环创建，不会是copy
## 二是用copy.deepcopy(list)创建一个新的deep copy样本

piles = [3,9,1,2]
n = len(piles)
dp = []
for i in range(n):
    d = [[0,0] for f in range(i)]+[[piles[i], 0]] + [[0,0] for f in range(n-i-1)]
    dp.append(d)  
dp

[[[3, 0], [0, 0], [0, 0], [0, 0]],
 [[0, 0], [9, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [1, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0], [2, 0]]]

In [51]:
for diff in range(1,n):
    for i in range(n-diff):
        j = diff + i
        ##以上是实现斜边遍历的循环，可以手写草稿找到diff和i的范围
        ## 先手选择左边石头堆或右边石头堆的两个数值，择优
        left = piles[i] + dp[i+1][j][1]
        right= piles[j] + dp[i][j-1][1]
        print(f"dp[{i}][{j}]", end = "\t")
        print(f"left:{left}\tright:{right}", end = "\t")
        if left > right:
            dp[i][j][0] = left
            dp[i][j][1] = dp[i+1][j][0]
        else:
            dp[i][j][0] = right
            dp[i][j][1] = dp[i][j-1][0]
        print(f"dp[{i}][{j}] = {dp[i][j]}")
        print(dp[i])
            
dp

dp[0][1]	left:3	right:9	dp[0][1] = [9, 3]
[[3, 0], [9, 3], [0, 0], [0, 0]]
dp[1][2]	left:9	right:1	dp[1][2] = [9, 1]
[[0, 0], [9, 0], [9, 1], [0, 0]]
dp[2][3]	left:1	right:2	dp[2][3] = [2, 1]
[[0, 0], [0, 0], [1, 0], [2, 1]]
dp[0][2]	left:4	right:4	dp[0][2] = [4, 9]
[[3, 0], [9, 3], [4, 9], [0, 0]]
dp[1][3]	left:10	right:3	dp[1][3] = [10, 2]
[[0, 0], [9, 0], [9, 1], [10, 2]]
dp[0][3]	left:5	right:11	dp[0][3] = [11, 4]
[[3, 0], [9, 3], [4, 9], [11, 4]]


[[[3, 0], [9, 3], [4, 9], [11, 4]],
 [[0, 0], [9, 0], [9, 1], [10, 2]],
 [[0, 0], [0, 0], [1, 0], [2, 1]],
 [[0, 0], [0, 0], [0, 0], [2, 0]]]

In [30]:
## 草拟实现斜边遍历
for diff in range(1,n):
    for i in range(n-diff):
        j = diff + i
        print(diff,i,j)

1 0 1
1 1 2
1 2 3
2 0 2
2 1 3
3 0 3


In [53]:
c = [1,4]
import numpy as np
np.diff(c)

array([3])